SPARSA Demo: Some Cool Python Tricks
====================================

In this set of examples, I'll show a few things that I've found interesting in Python. Most notably:
- \__str__
- \__call__
- generators
- function pointers



Class Demos
-----------

First, let's set up some classes, one with a \__str__ method, and one without

In [1]:
class DumbNode:
    """
    A simple object for comparison against the 'Node' object
    """
    def __init__(self):
        print("DumbNode object created!")
        
class Node:
    """
    A more interesting object that can be printed out, initialized in several
    different ways, and be called as a function.
    """
    def __init__(self, var1=1, var2=4):
        """
        Initializes the node. Has optional parameters for setting initial values
        :param var1: First parameter
        :param var2: Second Parameter
        :return: the Node object
        """
        self.p = var1
        self.q = var2
        print("Node object created!")

    def __str__(self):
        """
        Makes a human-readable string for custom objects
        :return: the string representation of the object
        """
        return "Node object with p=%d, and q=%d" % (self.p, self.q)

    def __call__(self, arg1, arg2):
        """
        Allows the object to be called as a function
        :param arg1: A number
        :param arg2: Another number
        :return: tuple containing the existing values plus the argument values
        """
        return arg1 + self.p, arg2 + self.q

Some interesting things about the *Node* class: it can be converted gracefully into a string, unlike *DumbNode*, it can be called as a function once it is initialized, and the input parameters don't need to be defined by the user if they do not need to change from the default values. A few examples are shown below.

In [2]:
# Let's create some objects
dd = DumbNode()
a = Node()
b = Node(9)  # var1=9, var2 gets default value
c = Node(0, 0)  # var1=0, var2=0
d = Node(var2=5, var1=4)
e = Node(var2=99)  # var2=99, var1 gets default value

# And now we can print them in plain text because of __str__
print(dd)  # prints out memory location. Python doesn't know how to make it a string.
print(a)  # Python can make these strings because of __str__
print(b)
print(c)
print(d)
print(a(7, 8))  # called object 'a' as a function. prints the result

DumbNode object created!
Node object created!
Node object created!
Node object created!
Node object created!
Node object created!
<__main__.DumbNode instance at 0x0000000003D73A88>
Node object with p=1, and q=4
Node object with p=9, and q=4
Node object with p=0, and q=0
Node object with p=4, and q=5
(8, 12)


The output from the nodes shows when they are created, and then what happens when the nodes are printed, especially after changing the parameters to the Node object constructor. Notice what happens when the \__str__ method is not defined by the user.

Generator Demos
---------------

In these sections, I'll show you have generators work and differ from standard functions. Let's use the Fibonacci Sequence as an example.

In [3]:
def fib_r(n=100):
    """
    Makes the Fibonacci sequence. Requires an upper bound on numbers
    :param n: Number of items in the sequence to determine
    :return: a list of the first n items in the Fibonacci sequence
    """
    num1 = 0
    num2 = 1
    counter = 2
    results = [num1, num2]
    while counter < n:
        f_sum = num1 + num2
        num1 = num2
        num2 = f_sum
        results.append(f_sum)
        counter += 1
    return results

def fib_g():
    """
    Makes the Fibonacci sequence as a generator. Does not need an upper bound
    :return: Yields the next number of the sequence when called
    """
    num1 = 0
    yield num1
    num2 = 1
    yield num2
    while True:
        f_sum = num1 + num2
        num1 = num2
        num2 = f_sum
        yield f_sum

There are two main differences between these functions. First, the first function makes a list and then returns it when an upper bound is reached. That entire list is then copied back to the user, but for large values of n, that gets to be unmanageable, especially if the middle values aren't necessary for the application. In the second function, yield statements are used to get a number back to the user. Each new number is only generated when the user asks for it, either in a next statement, or in a for-each loop.

In [4]:
count = 10
print("First Fib")
p = fib_r(count)  # fib_r needs upper bound because return is used
print(type(p))  # p is a list.
for i in range(count):
    num = p[i]
# Next line prints the last number in sequence. To get 11th number, list
# must be regenerated with n=11. This is memory intensive with large n.
print num

First Fib
<type 'list'>
34


Using the first function, which uses a return statement, a list of 10 numbers was returned. However, if the 11th number in the sequence is needed, the entire list must be regenerated by calling the function with a larger upper bound. If we needed the 9 millionth number in the sequence, we would have to return a list of 9 million numbers. Using Python's default size for an int (8 bytes), 9 million * 8 bytes = 72MB of additional space just to get that last number. Contrast that to the generator, using yield statements.

In [5]:
print("Second Fib")
q = fib_g()
print(type(q))  # q is a generator
for i in range(count):
    # next(q) gets next number in sequence. It's a manual way of getting a
    # value instead of using a for-in loop, which is supported
    num = next(q)
# Next line prints the same value generated from above, with the
# difference that more values can be generated with no additional function
# calls or memory usage than what is already used.
print num

Second Fib
<type 'generator'>
34


As you can see in this second example, the same results are achieved, but because the generator was used, no additional memory than was used in the function was used. Also, unlike the original function, this one has no upper bound. We can keep on generating numbers with no additional memory penalties.

In [6]:
print(next(q))  # 11th number
print(next(q))  # 12th number
print(next(q))  # 13th number

55
89
144


Also, in the case that we want to find the first number in a sequence larger than some value, we can do that without having to guess an upper limit on the sequence. Let's find the first number in the Fibonacci Sequence greater than 9 million.

In [7]:
for number in fib_g():  # example using the for-in loop with a generator
    # gets the first number in the Fibonacci sequence greater than 9,000,000
    # requires no knowledge of upper bounds and uses minimal memory
    if number > 9e6:
        print number
        break

9227465


Function Pointer Demos
----------------------

In Python, function pointers exist. Usually, it may be the case that you have to apply different parameters to the same function and iterate based on the values, but what if you had to apply the same data to many functions? Let's set up some simple mathematical functions.

In [8]:
def add(a, b):
    """
    Simple adder
    :param a:
    :param b:
    :return:
    """
    return a+b


def subtract(a, b):
    """
    Simple subtractor
    :param a:
    :param b:
    :return:
    """
    return a-b


def multiply(a, b):
    """
    Simple multiplier
    :param a:
    :param b:
    :return:
    """
    return a*b


def divide(a, b):
    """
    Simple divider
    :param a:
    :param b:
    :return:
    """
    return float(a)/float(b)

Now, we can set up a list of data to apply to the functions. The following line may look tricky, but here's the English translation for the Python code. Create a list of tuples of a number and 10 times the number for numbers in the set [1,10), counting by 1, and assign it to the variable, nums.

In [9]:
# make a list of number pairs. [(1.0, 10.0), ..., (9.0, 90.0)]
nums = [(float(i), float(i*10)) for i in range(1, 10, 1)]
print nums

[(1.0, 10.0), (2.0, 20.0), (3.0, 30.0), (4.0, 40.0), (5.0, 50.0), (6.0, 60.0), (7.0, 70.0), (8.0, 80.0), (9.0, 90.0)]


Now, we can make a list of functions to apply to the data!

In [10]:
# list of functions defined above
ops = [add, subtract, multiply, divide]

Now, using the data pairs and set of functions, we can apply everything to generate a 4x9 matrix of results of each operation. The rows will correspond to the operations performed, in order of the list order, and the columns will correspond to the index of the number pair used, in order from 1 to 9.

In [11]:
results = []
for function in ops:
    # makes 4x9 matrix: 4 rows for operations, 9 cols for each pair result
    # each function is called with the number pair as a parameter
    results.append([function(tup[0], tup[1]) for tup in nums])
for row in results:  # just prints out the matrix one row at a time
    print row  # in order: results of add, subtract, multiply, divide

[11.0, 22.0, 33.0, 44.0, 55.0, 66.0, 77.0, 88.0, 99.0]
[-9.0, -18.0, -27.0, -36.0, -45.0, -54.0, -63.0, -72.0, -81.0]
[10.0, 40.0, 90.0, 160.0, 250.0, 360.0, 490.0, 640.0, 810.0]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
